ovo je ukljuceno u prophet, ali treba li i kod ostalih ml modela? treba probat

for O_ features data (Temp, RH etc.) starts in 2017,
            # and not in 2010 like for other stations (AREAS)
            df_org = df_org.loc[a].copy()  # because of that, all data for O_ starts when features available 

optimisation of prophet parameters?

fill nans before prediction? han did it in prophet only with mean()... we in paper filled with bfill?

In [1]:
import sys
sys.path.append('../..')
from src.modules.models_new2 import *
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Read data from 01_DataProcessing

In [2]:
path = '../../results/5sta(DNOWS)_3com(NO2,PM10K,O3)_year2010_TemporalDummies_Ox_holidays_meteo.csv'
df_start = pd.read_csv(path, index_col=0).drop(columns=['N_Ox','S_Ox']) # exclude Ox
df_start.index = pd.to_datetime(df_start.index)
# Add feature holidays on sunday
df_start['holiday_sunday'] = holiday_sunday(df_start)
# Add feature month
df_start['month'] = month(df_start)
df_o = df_start.copy()
df_o

,D_NO2,D_PM10K,N_NO2,N_O3,N_PM10K,O_NO2,O_PM10K,S_NO2,S_O3,S_PM10K,...,West_Temp,West_Winddirection,West_Windspeed,Ost_Pressure,Ost_RH,Ost_Temp,Ost_Winddirection,Ost_Windspeed,holiday_sunday,month
2010-01-01,62.420425,48.681210,37.758297,4.000452,34.853405,52.362980,41.291916,45.165530,2.108327,58.876480,...,3.497470,189.95006,0.326422,NaN,NaN,NaN,NaN,NaN,0,1
2010-01-02,49.886177,148.307540,25.938934,4.637932,45.783848,36.956390,76.422780,33.300850,1.795727,169.943600,...,3.033653,214.33745,0.206639,NaN,NaN,NaN,NaN,NaN,0,1
2010-01-03,21.900616,21.664154,11.074873,34.357640,10.978717,15.286636,16.273163,14.216703,30.359385,19.864191,...,2.953737,278.73930,2.640419,NaN,NaN,NaN,NaN,NaN,0,1
2010-01-04,36.261917,21.462568,18.268064,48.127280,14.785930,24.183193,20.439781,26.544174,43.230330,26.680067,...,-0.571709,285.80966,1.023319,NaN,NaN,NaN,NaN,NaN,0,1
2010-01-05,71.757225,53.670230,41.781490,12.211806,32.000100,42.627240,43.426716,51.446163,7.382785,54.818146,...,-4.016794,198.60895,0.356867,NaN,NaN,NaN,NaN,NaN,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-07-03,33.136486,18.087683,14.935202,47.710407,18.953040,19.609596,14.622527,21.049612,36.083650,15.733115,...,18.413990,264.72040,0.624361,973.1446,73.206460,18.672241,182.59240,0.435133,0,7
2021-07-04,19.907900,14.338615,8.624179,78.167210,15.033513,14.503092,14.373650,13.824055,67.281296,14.214394,...,21.988039,179.25130,0.576749,972.4365,60.177140,22.199862,130.74823,0.562957,0,7
2021-07-05,20.364044,16.338556,16.112087,66.048890,17.262230,12.612704,15.058688,12.696744,70.662820,13.985309,...,20.624140,174.17639,0.545803,969.7183,71.621360,20.727047,139.74475,0.418278,0,7
2021-07-06,24.437843,19.290907,7.977263,71.018890,15.782174,15.851383,17.253944,13.073669,58.211086,14.010532,...,21.858849,198.03795,0.677458,969.6665,74.613620,21.856710,129.45146,0.462159,0,7


# Prediction

Cutoff = 0.9

In [3]:
s1 = '2020-01-03'
e1 = '2020-03-10'
percent = 0.9
randomstate = 111
others = ['month', 'holiday', 'holiday_school', 'holiday_sunday']

## Random Forest prediction

In [6]:
params_gbm ={
    'max_depth':(1, 10),
    'n_estimators':(30, 500)
}

In [8]:
%%time
df_pred = rf_prediction(df_o=df_o, percent=percent, randomstate=randomstate, params_gbm=params_gbm, factors=factors, 
                        add_feat=others)

NO2
DonBosco_ D_
|   iter    |  target   | max_depth | n_esti... |
-------------------------------------------------
|  1        |  0.4663   |  6.51     |  109.5    |
|  2        |  0.4419   |  4.925    |  391.6    |
|  3        |  0.405    |  3.658    |  100.1    |
|  4        |  0.1412   |  1.202    |  227.5    |
|  5        |  0.3567   |  3.148    |  188.7    |
|  6        |  0.459    |  9.916    |  141.7    |
|  7        |  0.2745   |  1.731    |  344.7    |
|  8        |  0.4662   |  6.591    |  158.9    |
|  9        |  0.4398   |  5.196    |  85.63    |
|  10       |  0.2744   |  1.666    |  453.4    |
|  11       |  0.4564   |  9.823    |  55.37    |
|  12       |  0.1354   |  1.0      |  30.0     |
|  13       |  0.4665   |  7.849    |  109.8    |
|  14       |  0.4592   |  10.0     |  500.0    |
|  15       |  0.4644   |  9.068    |  290.4    |
{'max_depth': 8, 'n_estimators': 110, 'random_state': 111}
Nord_ N_
|   iter    |  target   | max_depth | n_esti... |
---------------

|  11       |  0.3938   |  9.887    |  56.5     |
|  12       |  0.1785   |  1.0      |  30.0     |
|  13       |  0.4024   |  7.582    |  158.0    |
|  14       |  0.3984   |  10.0     |  500.0    |
|  15       |  0.3985   |  10.0     |  289.1    |
{'max_depth': 8, 'n_estimators': 158, 'random_state': 111}
Nord_ N_
|   iter    |  target   | max_depth | n_esti... |
-------------------------------------------------
|  1        |  0.3758   |  6.51     |  109.5    |
|  2        |  0.3344   |  4.925    |  391.6    |
|  3        |  0.2876   |  3.658    |  100.1    |
|  4        |  0.08946  |  1.202    |  227.5    |
|  5        |  0.2234   |  3.148    |  188.7    |
|  6        |  0.3942   |  9.916    |  141.7    |
|  7        |  0.1527   |  1.731    |  344.7    |
|  8        |  0.3755   |  6.591    |  158.9    |
|  9        |  0.3339   |  5.196    |  85.63    |
|  10       |  0.1527   |  1.666    |  453.4    |
|  11       |  0.08907  |  1.207    |  126.2    |
|  12       |  0.3927   |  8.687

|  5        |  0.7867   |  3.148    |  188.7    |
|  6        |  0.8689   |  9.916    |  141.7    |
|  7        |  0.675    |  1.731    |  344.7    |
|  8        |  0.8635   |  6.591    |  158.9    |
|  9        |  0.8466   |  5.196    |  85.63    |
|  10       |  0.6751   |  1.666    |  453.4    |
|  11       |  0.8674   |  9.892    |  40.01    |
|  12       |  0.8692   |  9.299    |  499.9    |
|  13       |  0.8675   |  7.757    |  499.8    |
|  14       |  0.5168   |  1.0      |  59.88    |
|  15       |  0.5165   |  1.0      |  126.6    |
{'max_depth': 9, 'n_estimators': 500, 'random_state': 111}
West_ W_
Wall time: 35min 2s


In [9]:
df_pred

,D_NO2,D_NO2_pred,N_NO2,N_NO2_pred,O_NO2,O_NO2_pred,S_NO2,S_NO2_pred,W_NO2,W_NO2_pred,...,O_PM10K,O_PM10K_pred,S_PM10K,S_PM10K_pred,W_PM10K,W_PM10K_pred,N_O3,N_O3_pred,S_O3,S_O3_pred
2020-01-03,56.696457,58.632652,39.334070,38.490673,44.132984,42.080242,42.729170,49.302271,41.950268,44.575613,...,46.388460,34.913698,56.982330,52.255321,45.637370,39.344855,9.617968,20.209084,3.537352,6.915091
2020-01-04,66.766790,58.835236,43.032360,37.943533,47.897724,42.136814,48.308884,46.052183,49.090330,44.714654,...,37.184284,34.872038,57.270626,53.076709,38.993320,39.595008,6.617978,15.438404,1.221737,10.157322
2020-01-05,52.037860,57.057979,35.747350,33.440684,42.127870,41.311235,39.910446,48.220324,42.618286,43.034291,...,35.905117,37.045224,51.025585,47.366805,38.226850,31.145058,20.701195,23.733940,4.793549,8.971175
2020-01-06,32.097057,36.005107,13.998880,20.674946,21.196960,23.733125,20.614933,19.726656,17.250006,21.196471,...,16.606459,24.796482,13.068683,18.924878,12.079635,16.799884,55.537983,41.095946,46.128056,43.136378
2020-01-07,49.639480,60.289147,30.543589,35.574938,37.692710,43.405082,36.684887,50.255919,37.704323,44.643267,...,30.604795,33.890909,32.939518,50.672748,25.352188,37.013905,22.510405,20.563603,13.748825,9.398436
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-03-06,47.934826,57.010615,23.931540,24.684927,23.563103,38.896325,29.860310,32.837648,30.248318,38.253223,...,19.673954,33.356379,22.934511,29.089321,18.844595,27.573340,47.858826,52.199077,38.594560,46.456389
2020-03-07,54.893112,56.473422,30.731730,25.521993,31.140863,34.022219,36.039930,45.095294,33.999302,35.161040,...,14.401324,30.234672,15.170040,36.751029,11.644395,30.724495,31.879847,36.407183,29.409145,24.415709
2020-03-08,39.672287,43.883177,18.303322,22.765905,19.777733,34.632241,22.489891,36.636955,26.882597,29.683811,...,18.801638,34.066318,18.809082,26.203612,15.616488,25.064877,40.864746,48.904296,32.988370,39.058197
2020-03-09,37.320225,39.176748,16.569525,19.186513,21.383870,33.411034,27.240894,37.218146,29.120990,22.949427,...,17.584291,32.142267,17.909945,31.960157,17.079546,25.604304,48.081580,51.300126,35.395940,31.860974


In [11]:
df_pred.to_csv('../../results/models/rf_04_20200103_20200310.csv')

### Train for station O_ from 2017 because then weather features start and compare prediction outcome

In [12]:
df_o.columns

Index(['D_NO2', 'D_PM10K', 'N_NO2', 'N_O3', 'N_PM10K', 'O_NO2', 'O_PM10K',
       'S_NO2', 'S_O3', 'S_PM10K', 'W_NO2', 'W_PM10K', 'year', 'dayofyear',
       'month_Apr', 'month_Aug', 'month_Dec', 'month_Feb', 'month_Jan',
       'month_Jul', 'month_Jun', 'month_Mar', 'month_May', 'month_Nov',
       'month_Oct', 'month_Sep', 'weekday_Friday', 'weekday_Monday',
       'weekday_Saturday', 'weekday_Sunday', 'weekday_Thursday',
       'weekday_Tuesday', 'weekday_Wednesday', 'season_fall', 'season_spring',
       'season_summer', 'season_winter', 'holiday', 'holiday_school',
       'DonBosco_RH', 'DonBosco_Temp', 'Nord_Precip', 'Nord_Pressure',
       'Nord_RH', 'Nord_Temp', 'Nord_Winddirection', 'Nord_Windspeed',
       'Sud_RH', 'Sud_Temp', 'Sud_Winddirection', 'Sud_Windspeed', 'West_RH',
       'West_Temp', 'West_Winddirection', 'West_Windspeed', 'Ost_Pressure',
       'Ost_RH', 'Ost_Temp', 'Ost_Winddirection', 'Ost_Windspeed',
       'holiday_sunday', 'month'],
      dtype='object')

In [33]:
df_o_O = df_o['2017-05-19':]
df_o_O

,D_NO2,D_PM10K,N_NO2,N_O3,N_PM10K,O_NO2,O_PM10K,S_NO2,S_O3,S_PM10K,...,West_Temp,West_Winddirection,West_Windspeed,Ost_Pressure,Ost_RH,Ost_Temp,Ost_Winddirection,Ost_Windspeed,holiday_sunday,month
2017-05-19,45.639910,24.332872,16.957468,78.190170,30.743067,25.911076,22.751585,23.654951,71.316970,22.146965,...,18.704782,165.69081,0.653614,974.42645,53.720432,18.787678,178.26420,0.542665,0,5
2017-05-20,42.444412,25.740637,15.916371,84.488450,24.494043,25.942177,24.831266,27.920233,75.111630,23.527933,...,20.344995,166.47047,0.787654,968.94380,49.370285,20.372522,172.34846,0.438690,0,5
2017-05-21,14.411227,25.637629,3.068115,85.255900,16.989332,7.083466,12.104981,6.569948,84.231070,13.862050,...,18.264944,324.59158,2.865858,972.46260,38.401066,18.327272,315.75570,1.388386,0,5
2017-05-22,12.642211,16.931640,2.806221,86.177620,11.698574,5.763199,11.096896,5.581210,84.809080,9.926031,...,16.927965,303.03060,2.439139,977.37570,51.819256,17.033990,284.92160,1.322250,0,5
2017-05-23,36.671116,24.818975,11.061803,82.107635,16.562471,21.624190,18.448933,19.599410,70.779750,20.905146,...,17.814220,154.77286,0.868903,975.57690,60.952880,17.525282,162.69939,0.569737,0,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-07-03,33.136486,18.087683,14.935202,47.710407,18.953040,19.609596,14.622527,21.049612,36.083650,15.733115,...,18.413990,264.72040,0.624361,973.14460,73.206460,18.672241,182.59240,0.435133,0,7
2021-07-04,19.907900,14.338615,8.624179,78.167210,15.033513,14.503092,14.373650,13.824055,67.281296,14.214394,...,21.988039,179.25130,0.576749,972.43650,60.177140,22.199862,130.74823,0.562957,0,7
2021-07-05,20.364044,16.338556,16.112087,66.048890,17.262230,12.612704,15.058688,12.696744,70.662820,13.985309,...,20.624140,174.17639,0.545803,969.71830,71.621360,20.727047,139.74475,0.418278,0,7
2021-07-06,24.437843,19.290907,7.977263,71.018890,15.782174,15.851383,17.253944,13.073669,58.211086,14.010532,...,21.858849,198.03795,0.677458,969.66650,74.613620,21.856710,129.45146,0.462159,0,7


In [40]:
%%time
df_O_pred = rf_prediction_Ost(df_o=df_o_O, percent=percent, randomstate=randomstate, params_gbm=params_gbm, 
                              factors=['NO2','PM10K'], add_feat=others)

NO2
Ost_ O_
|   iter    |  target   | max_depth | n_esti... |
-------------------------------------------------
|  1        |  0.5322   |  6.51     |  109.5    |
|  2        |  0.5138   |  4.925    |  391.6    |
|  3        |  0.4964   |  3.658    |  100.1    |
|  4        |  0.1776   |  1.202    |  227.5    |
|  5        |  0.4195   |  3.148    |  188.7    |
|  6        |  0.5319   |  9.916    |  141.7    |
|  7        |  0.3264   |  1.731    |  344.7    |
|  8        |  0.5282   |  6.591    |  158.9    |
|  9        |  0.5136   |  5.196    |  85.63    |
|  10       |  0.3267   |  1.666    |  453.4    |
|  11       |  0.5361   |  9.981    |  37.66    |
|  12       |  0.1723   |  1.0      |  58.31    |
|  13       |  0.5301   |  9.209    |  36.54    |
|  14       |  0.1737   |  1.0      |  126.4    |
|  15       |  0.1749   |  1.0      |  149.4    |
{'max_depth': 10, 'n_estimators': 38, 'random_state': 111}
PM10K
Ost_ O_
|   iter    |  target   | max_depth | n_esti... |
---------------

* ### Same either if nans from 2010 to 2017 included or excluded

In [44]:
display(df_pred[['O_NO2_pred']])
display(df_O_pred[['O_NO2_pred']])

,O_NO2_pred
2020-01-03,42.080242
2020-01-04,42.136814
2020-01-05,41.311235
2020-01-06,23.733125
2020-01-07,43.405082
...,...
2020-03-06,38.896325
2020-03-07,34.022219
2020-03-08,34.632241
2020-03-09,33.411034


,O_NO2_pred
2020-01-03,42.080242
2020-01-04,42.136814
2020-01-05,41.311235
2020-01-06,23.733125
2020-01-07,43.405082
...,...
2020-03-06,38.896325
2020-03-07,34.022219
2020-03-08,34.632241
2020-03-09,33.411034


In [53]:
print(r2_score(df_pred['N_NO2'], df_pred['N_NO2_pred']))

0.6072572710239217


In [54]:
print(r2_score(df_pred['O_NO2'], df_pred['O_NO2_pred']))
print(r2_score(df_O_pred['O_NO2'], df_O_pred['O_NO2_pred']))

0.5264353759777303
0.5264353759777303


* ### Fill nans with bfill

In [90]:
# nans present only in Ost_ weather features

print(df_o.isnull().sum()[-8:])
print(df_o['2017-05-19':].isnull().sum()[-8:])

West_Windspeed          0
Ost_Pressure         2701
Ost_RH               2706
Ost_Temp             2702
Ost_Winddirection    2701
Ost_Windspeed        2701
holiday_sunday          0
month                   0
dtype: int64
West_Windspeed        0
Ost_Pressure          6
Ost_RH               11
Ost_Temp              7
Ost_Winddirection     6
Ost_Windspeed         6
holiday_sunday        0
month                 0
dtype: int64


In [56]:
df_O_fill = df_o_O.fillna(method='bfill')
df_O_fill.isnull().sum().sum()

0

In [57]:
%%time
df_O_fill_pred = rf_prediction_Ost(df_o=df_O_fill, percent=percent, randomstate=randomstate, params_gbm=params_gbm, 
                                   factors=['NO2','PM10K'], add_feat=others)

NO2
Ost_ O_
|   iter    |  target   | max_depth | n_esti... |
-------------------------------------------------
|  1        |  0.5408   |  6.51     |  109.5    |
|  2        |  0.5265   |  4.925    |  391.6    |
|  3        |  0.5101   |  3.658    |  100.1    |
|  4        |  0.185    |  1.202    |  227.5    |
|  5        |  0.4277   |  3.148    |  188.7    |
|  6        |  0.5432   |  9.916    |  141.7    |
|  7        |  0.3264   |  1.731    |  344.7    |
|  8        |  0.5427   |  6.591    |  158.9    |
|  9        |  0.5225   |  5.196    |  85.63    |
|  10       |  0.3263   |  1.666    |  453.4    |
|  11       |  0.5219   |  9.772    |  32.18    |
|  12       |  0.1795   |  1.0      |  56.62    |
|  13       |  0.5424   |  9.964    |  140.3    |
|  14       |  0.5378   |  10.0     |  413.8    |
|  15       |  0.5391   |  9.42     |  288.9    |
{'max_depth': 10, 'n_estimators': 142, 'random_state': 111}
PM10K
Ost_ O_
|   iter    |  target   | max_depth | n_esti... |
--------------

* #### In case of NO2 bfill contributes to better r2 score, but in case of PM10K not

In [58]:
print(r2_score(df_O_pred['O_NO2'], df_O_pred['O_NO2_pred']))
print(r2_score(df_O_fill_pred['O_NO2'], df_O_fill_pred['O_NO2_pred']))

0.5264353759777303
0.5022400034355663


In [59]:
print(r2_score(df_O_pred['O_PM10K'], df_O_pred['O_PM10K_pred']))
print(r2_score(df_O_fill_pred['O_PM10K'], df_O_fill_pred['O_PM10K_pred']))

0.19224953034166403
0.1953672870587463


> ### Han used fill method only in prophet, she used mean()

## Extra Trees prediction

In [4]:
params_gbm ={
    'max_depth':(2, 10),
    'n_estimators':(50, 500)
}

In [5]:
%%time
df_pred_et = extree_prediction(df_o=df_o, percent=percent, randomstate=randomstate, params_gbm=params_gbm, 
                               factors=factors, add_feat=others)

NO2
DonBosco_ D_
|   iter    |  target   | max_depth | n_esti... |
-------------------------------------------------
|  1        |  0.4522   |  6.897    |  126.1    |
|  2        |  0.4186   |  5.488    |  396.2    |
|  3        |  0.3906   |  4.363    |  117.1    |
|  4        |  0.2594   |  2.18     |  239.1    |
|  5        |  0.3912   |  3.909    |  201.9    |
|  6        |  0.4604   |  9.926    |  157.0    |
|  7        |  0.3362   |  2.65     |  351.3    |
|  8        |  0.4522   |  6.97     |  173.4    |
|  9        |  0.4397   |  5.73     |  103.3    |
|  10       |  0.3367   |  2.592    |  455.3    |
|  11       |  0.4593   |  8.332    |  140.7    |
|  12       |  0.4605   |  10.0     |  84.43    |
|  13       |  0.4592   |  7.602    |  136.1    |
|  14       |  0.2602   |  2.0      |  59.34    |
|  15       |  0.4627   |  9.171    |  299.4    |
{'max_depth': 9, 'n_estimators': 299, 'random_state': 111}
Nord_ N_
|   iter    |  target   | max_depth | n_esti... |
---------------

|  11       |  0.4017   |  9.903    |  313.6    |
|  12       |  0.3862   |  6.892    |  125.8    |
|  13       |  0.2157   |  2.049    |  162.9    |
|  14       |  0.4023   |  9.493    |  152.2    |
|  15       |  0.2698   |  3.396    |  312.2    |
{'max_depth': 9, 'n_estimators': 152, 'random_state': 111}
Nord_ N_
|   iter    |  target   | max_depth | n_esti... |
-------------------------------------------------
|  1        |  0.3406   |  6.897    |  126.1    |
|  2        |  0.2955   |  5.488    |  396.2    |
|  3        |  0.2572   |  4.363    |  117.1    |
|  4        |  0.14     |  2.18     |  239.1    |
|  5        |  0.2581   |  3.909    |  201.9    |
|  6        |  0.3727   |  9.926    |  157.0    |
|  7        |  0.2022   |  2.65     |  351.3    |
|  8        |  0.343    |  6.97     |  173.4    |
|  9        |  0.3258   |  5.73     |  103.3    |
|  10       |  0.2022   |  2.592    |  455.3    |
|  11       |  0.3755   |  9.903    |  313.6    |
|  12       |  0.3406   |  6.892

|  5        |  0.8195   |  3.909    |  201.9    |
|  6        |  0.8753   |  9.926    |  157.0    |
|  7        |  0.7806   |  2.65     |  351.3    |
|  8        |  0.8651   |  6.97     |  173.4    |
|  9        |  0.856    |  5.73     |  103.3    |
|  10       |  0.7806   |  2.592    |  455.3    |
|  11       |  0.875    |  9.903    |  313.6    |
|  12       |  0.865    |  6.892    |  125.8    |
|  13       |  0.7365   |  2.049    |  162.9    |
|  14       |  0.873    |  9.493    |  152.2    |
|  15       |  0.7815   |  3.174    |  312.8    |
{'max_depth': 10, 'n_estimators': 157, 'random_state': 111}
West_ W_
Wall time: 7min 29s


In [6]:
df_pred_et

,D_NO2,D_NO2_pred,N_NO2,N_NO2_pred,O_NO2,O_NO2_pred,S_NO2,S_NO2_pred,W_NO2,W_NO2_pred,...,O_PM10K,O_PM10K_pred,S_PM10K,S_PM10K_pred,W_PM10K,W_PM10K_pred,N_O3,N_O3_pred,S_O3,S_O3_pred
2020-01-03,56.696457,58.026188,39.334070,37.883876,44.132984,42.258048,42.729170,47.350485,41.950268,43.502377,...,46.388460,36.770222,56.982330,51.262967,45.637370,37.821561,9.617968,14.715434,3.537352,7.645473
2020-01-04,66.766790,57.170343,43.032360,36.369568,47.897724,41.555265,48.308884,45.790084,49.090330,43.329208,...,37.184284,35.183993,57.270626,51.573477,38.993320,39.289788,6.617978,17.707933,1.221737,8.358451
2020-01-05,52.037860,54.777618,35.747350,32.593179,42.127870,40.349204,39.910446,45.019738,42.618286,41.534516,...,35.905117,35.852228,51.025585,45.680741,38.226850,32.693411,20.701195,20.567792,4.793549,8.047600
2020-01-06,32.097057,31.549744,13.998880,22.977939,21.196960,20.880264,20.614933,20.448572,17.250006,18.575657,...,16.606459,20.900321,13.068683,19.192682,12.079635,14.632802,55.537983,39.548610,46.128056,42.153287
2020-01-07,49.639480,59.776983,30.543589,37.368761,37.692710,42.241048,36.684887,49.376257,37.704323,43.659361,...,30.604795,32.188495,32.939518,49.926939,25.352188,37.450310,22.510405,21.279772,13.748825,10.843531
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-03-06,47.934826,55.161074,23.931540,27.467179,23.563103,37.983324,29.860310,35.142950,30.248318,37.193441,...,19.673954,32.675474,22.934511,29.635453,18.844595,30.250200,47.858826,51.233499,38.594560,43.711116
2020-03-07,54.893112,55.528062,30.731730,26.839710,31.140863,38.282235,36.039930,43.046959,33.999302,36.829479,...,14.401324,31.270548,15.170040,39.096558,11.644395,31.017703,31.879847,32.836529,29.409145,27.477937
2020-03-08,39.672287,43.880561,18.303322,22.149112,19.777733,28.215574,22.489891,28.082431,26.882597,29.579380,...,18.801638,29.202349,18.809082,24.646567,15.616488,25.701915,40.864746,45.289810,32.988370,41.483203
2020-03-09,37.320225,42.692643,16.569525,19.431390,21.383870,27.868152,27.240894,32.681546,29.120990,23.447479,...,17.584291,30.629691,17.909945,31.154381,17.079546,24.691846,48.081580,47.522479,35.395940,36.194702


In [7]:
df_pred_et.to_csv('../../results/models/extree_04_20200103_20200310.csv')

### XGBoost prediction

In [8]:
params_gbm ={
    'max_depth':(3, 10),
    'n_estimators':(50, 300),
    'eta': (0.008, 0.1),
    'subsample': (0.7, 1),
    'colsample_bytree': (0.7, 1)    
}

In [9]:
%%time
df_pred_xgb = xgb_prediction(df_o=df_o, percent=percent, randomstate=randomstate, params_gbm=params_gbm, 
                             factors=factors, add_feat=others)

NO2
DonBosco_ D_
|   iter    |  target   | colsam... |    eta    | max_depth | n_esti... | subsample |
-------------------------------------------------------------------------------------
|  1        |  0.4777   |  0.8837   |  0.02355  |  6.052    |  242.3    |  0.7886   |
|  2        | -0.901    |  0.7447   |  0.01007  |  5.942    |  109.7    |  0.8013   |
|  3        |  0.4823   |  0.9972   |  0.02987  |  3.568    |  217.4    |  0.8864   |
|  4        |  0.4623   |  0.7823   |  0.05089  |  3.829    |  68.49    |  0.9702   |
|  5        |  0.4069   |  0.9382   |  0.08533  |  8.706    |  297.7    |  0.8732   |
|  6        |  0.4856   |  0.9441   |  0.04676  |  3.192    |  163.5    |  0.7316   |
|  7        |  0.4529   |  0.9452   |  0.07219  |  6.957    |  118.6    |  0.9995   |
|  8        |  0.4705   |  0.7414   |  0.06462  |  6.395    |  151.3    |  0.9184   |
|  9        |  0.4733   |  0.7968   |  0.04485  |  5.233    |  286.8    |  0.9756   |
|  10       |  0.437    |  0.9442   |

|  4        |  0.7239   |  0.7823   |  0.05089  |  3.829    |  68.49    |  0.9702   |
|  5        |  0.7361   |  0.9382   |  0.08533  |  8.706    |  297.7    |  0.8732   |
|  6        |  0.7492   |  0.9441   |  0.04676  |  3.192    |  163.5    |  0.7316   |
|  7        |  0.745    |  0.9452   |  0.07219  |  6.957    |  118.6    |  0.9995   |
|  8        |  0.755    |  0.7414   |  0.06462  |  6.395    |  151.3    |  0.9184   |
|  9        |  0.7571   |  0.7968   |  0.04485  |  5.233    |  286.8    |  0.9756   |
|  10       |  0.7233   |  0.9442   |  0.01114  |  9.604    |  287.6    |  0.942    |
|  11       |  0.7508   |  0.8516   |  0.06249  |  6.508    |  157.8    |  0.9071   |
|  12       |  0.7357   |  1.0      |  0.1      |  7.785    |  125.8    |  1.0      |
|  13       |  0.7566   |  0.9464   |  0.05098  |  5.083    |  232.6    |  0.8396   |
|  14       |  0.7372   |  0.9111   |  0.04728  |  9.61     |  224.3    |  0.9672   |
|  15       |  0.7585   |  0.8474   |  0.05715  |  3.7

|  8        |  0.3947   |  0.7414   |  0.06462  |  6.395    |  151.3    |  0.9184   |
|  9        |  0.3985   |  0.7968   |  0.04485  |  5.233    |  286.8    |  0.9756   |
|  10       |  0.3334   |  0.9442   |  0.01114  |  9.604    |  287.6    |  0.942    |
|  11       |  0.383    |  0.8428   |  0.06698  |  4.819    |  157.3    |  0.8445   |
|  12       |  0.3509   |  1.0      |  0.1      |  7.795    |  126.0    |  1.0      |
|  13       |  0.3748   |  0.7118   |  0.07692  |  3.085    |  279.1    |  0.8611   |
|  14       |  0.3899   |  0.7937   |  0.01781  |  5.153    |  233.1    |  0.9481   |
|  15       |  0.3246   |  0.9189   |  0.09819  |  9.611    |  224.5    |  0.7      |
{'colsample_bytree': 0.7968441776453169, 'eta': 0.04485019059234422, 'max_depth': 5, 'n_estimators': 287, 'subsample': 0.9756125086793304, 'random_state': 111}
Ost_ O_
|   iter    |  target   | colsam... |    eta    | max_depth | n_esti... | subsample |
----------------------------------------------------------

|  12       |  0.5028   |  1.0      |  0.1      |  7.818    |  126.2    |  1.0      |
|  13       |  0.5421   |  0.9442   |  0.02737  |  4.482    |  226.3    |  0.8748   |
|  14       |  0.5112   |  0.8245   |  0.04123  |  9.888    |  234.0    |  0.8012   |
|  15       |  0.5026   |  0.8187   |  0.0643   |  9.769    |  251.0    |  0.9595   |
{'colsample_bytree': 0.9972137393892636, 'eta': 0.029870833758802097, 'max_depth': 4, 'n_estimators': 217, 'subsample': 0.8863728758202091, 'random_state': 111}
O3
DonBosco_ D_
Nord_ N_
|   iter    |  target   | colsam... |    eta    | max_depth | n_esti... | subsample |
-------------------------------------------------------------------------------------
|  1        |  0.8479   |  0.8837   |  0.02355  |  6.052    |  242.3    |  0.7886   |
|  2        |  0.4302   |  0.7447   |  0.01007  |  5.942    |  109.7    |  0.8013   |
|  3        |  0.8451   |  0.9972   |  0.02987  |  3.568    |  217.4    |  0.8864   |
|  4        |  0.8353   |  0.7823   |  0

In [10]:
df_pred_xgb

,D_NO2,D_NO2_pred,N_NO2,N_NO2_pred,O_NO2,O_NO2_pred,S_NO2,S_NO2_pred,W_NO2,W_NO2_pred,...,O_PM10K,O_PM10K_pred,S_PM10K,S_PM10K_pred,W_PM10K,W_PM10K_pred,N_O3,N_O3_pred,S_O3,S_O3_pred
2020-01-03,56.696457,58.232967,39.334070,37.599590,44.132984,41.677582,42.729170,47.978519,41.950268,44.349907,...,46.388460,36.751652,56.982330,53.063763,45.637370,42.880161,9.617968,16.032145,3.537352,8.527939
2020-01-04,66.766790,57.416100,43.032360,37.767052,47.897724,41.316528,48.308884,45.318996,49.090330,44.823936,...,37.184284,35.019836,57.270626,54.459988,38.993320,42.994595,6.617978,13.629521,1.221737,10.786489
2020-01-05,52.037860,52.024067,35.747350,34.153629,42.127870,39.222355,39.910446,42.061832,42.618286,42.623196,...,35.905117,34.279591,51.025585,47.206806,38.226850,32.642536,20.701195,21.653633,4.793549,11.977212
2020-01-06,32.097057,37.741718,13.998880,13.124260,21.196960,17.706100,20.614933,15.479812,17.250006,19.907465,...,16.606459,21.992460,13.068683,17.808249,12.079635,14.374945,55.537983,41.843884,46.128056,42.704498
2020-01-07,49.639480,59.932793,30.543589,34.609558,37.692710,44.384796,36.684887,51.424969,37.704323,44.196484,...,30.604795,39.973152,32.939518,50.563694,25.352188,38.309109,22.510405,21.792288,13.748825,8.516523
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-03-06,47.934826,56.059303,23.931540,25.391521,23.563103,41.482601,29.860310,33.315067,30.248318,39.702240,...,19.673954,35.529003,22.934511,28.516088,18.844595,28.182356,47.858826,49.954964,38.594560,46.759441
2020-03-07,54.893112,55.315025,30.731730,25.427376,31.140863,37.014370,36.039930,43.331745,33.999302,36.574726,...,14.401324,32.311218,15.170040,40.304440,11.644395,33.040554,31.879847,34.379959,29.409145,23.955492
2020-03-08,39.672287,47.374420,18.303322,23.113420,19.777733,33.869976,22.489891,33.666924,26.882597,30.938717,...,18.801638,30.747158,18.809082,24.174662,15.616488,24.761396,40.864746,43.939098,32.988370,40.803951
2020-03-09,37.320225,40.010208,16.569525,19.984650,21.383870,33.788544,27.240894,37.672676,29.120990,25.257210,...,17.584291,31.832073,17.909945,28.813383,17.079546,25.095110,48.081580,47.960026,35.395940,35.472187


In [11]:
df_pred_xgb.to_csv('../../results/models/xgb_04_20200103_20200310.csv')

## Prophet prediction

In [12]:
%%time
df_pred_pr = prophet_prediction(df_o=df_o, percent=percent, s1=s1, e1=e1, factors=factors)

f0_add
D_


INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


N_


INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


O_
(4206, 62)


...

W_


INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


f6
D_
N_


INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


O_
(4206, 89)
S_
W_
Wall time: 15min 34s


In [13]:
df_pred_pr

f0_add                                                         \
                D_NO2                 N_NO2                 O_NO2              
                 yhat          y       yhat          y       yhat          y   
ds                                                                             
2020-01-03  47.128515  56.696457  30.687054  39.334070  34.816279  44.132984   
2020-01-04  46.989032  64.316290  30.132661  40.563717  34.138211  47.122957   
2020-01-05  37.807865  52.037860  26.071201  35.747350  28.100080  42.127870   
2020-01-06  32.452076  32.097057  23.284748  13.998880  23.937769  21.196960   
2020-01-07  46.433365  49.639480  29.938373  30.543589  33.355029  37.692710   
...               ...        ...        ...        ...        ...        ...   
2020-03-06  48.864831  47.934826  25.807885  23.931540  32.401989  23.563103   
2020-03-07  48.658002  54.893112  25.081472  30.731730  31.602636  31.140863   
2020-03-08  39.331564  39.672287  20.820188  18.303322  25.391207  19.777733   
2020-03-09  33.753154  37.320225  17.806141  16.569525  21.003227  21.383870   
2020-03-10  47.436633  59.289640  24.204920  35.471115  30.143107  36.505016   

                                                        ...         f6  \
                S_NO2                 W_NO2             ...    O_PM10K   
                 yhat          y       yhat          y  ...       yhat   
ds                                                      ...              
2020-01-03  36.124008  42.729170  33.171985  41.950268  ...  35.086108   
2020-01-04  35.659437  48.308884  32.907099  45.323261  ...  33.644695   
2020-01-05  29.625064  39.910446  28.218164  42.618286  ...  24.894631   
2020-01-06  25.992454  20.614933  24.693789  17.250006  ...  15.074028   
2020-01-07  35.156644  36.684887  32.285526  37.704323  ...  29.186202   
...               ...        ...        ...        ...  ...        ...   
2020-03-06  34.549795  29.860310  30.525280  30.248318  ...  26.994410   
2020-03-07  33.991462  36.039930  30.262696  33.999302  ...  27.969530   
2020-03-08  27.814244  22.489891  25.540338  26.882597  ...  25.226639   
2020-03-09  23.989539  27.240894  21.944279  29.120990  ...  27.084203   
2020-03-10  32.913111  45.224900  29.424184  45.323261  ...  28.202052   

                                                                              \
                         S_PM10K               W_PM10K                  N_O3   
                    y       yhat          y       yhat          y       yhat   
ds                                                                             
2020-01-03  46.388460  42.212625  53.990230  34.473859  43.902282  16.645393   
2020-01-04  37.184284  42.392054  53.990230  34.279589  38.993320  14.480022   
2020-01-05  35.905117  33.704373  51.025585  26.014194  38.226850  19.518962   
2020-01-06  16.606459  18.700551  13.068683  14.505055  12.079635  33.281736   
2020-01-07  30.604795  37.231158  32.939518  28.656175  25.352188  24.694846   
...               ...        ...        ...        ...        ...        ...   
2020-03-06  19.673954  25.620861  22.934511  26.667950  18.844595  53.372648   
2020-03-07  14.401324  31.345820  15.170040  28.396818  11.644395  38.666333   
2020-03-08  18.801638  22.928258  18.809082  23.145052  15.616488  50.543773   
2020-03-09  17.584291  27.226882  17.909945  22.955578  17.079546  55.202003   
2020-03-10  32.242290  34.614791  37.680756  28.248330  27.224740  53.346103   

                                             
                            S_O3             
                    y       yhat          y  
ds                                           
2020-01-03   9.617968   8.371727   3.537352  
2020-01-04   6.617978   8.059930   1.221737  
2020-01-05  20.701195  14.193257   4.793549  
2020-01-06  55.537983  40.974447  46.128056  
2020-01-07  22.510405  16.745741  13.748825  
...               ...        ...        ...  
2020-03-06  47.858826  45.649824  38.59456

In [14]:
df_pred_pr.to_csv('../../results/models/prophet_04_20200103_20200310.csv')